In [1]:
import pickle

with open('/home/arplab/project/paradeigma/multi_modal/model/data/paradeigma_ts_20.pkl', 'rb') as f:
    annotation_20 = pickle.load(f)

In [15]:
import torch
import sklearn.svm as svm
import sklearn.metrics as mt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler

/home/arplab/project/paradeigma/multi_modal/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [54]:
# 패딩 함수

def sequence_padding(ts_list, padding_length = 50, mode = 'constant'):
    
    padding_value=0
    
    if (type(ts_list) != type([])) :
        ts_list = [padding_value] * padding_length
    
    elif len(ts_list) >= padding_length :
        ts_list = ts_list[0:padding_length]
    
    elif mode == 'constant':
        length = padding_length - len(ts_list)
        extend_list = [padding_value] * length
        ts_list = ts_list + extend_list    
        
    elif mode == 'replicate':
        
        quotient = padding_length // len(ts_list)
        remainder = padding_length % len(ts_list)
        result = ts_list * quotient
        result += ts_list[:remainder]
        ts_list = result
        
    return ts_list 

def make_emotion_vector(emotion):
    emotion_dict = {'neutral': 0, 'happy': 1, 'surprise':2, 'disqust': 3, 'angry': 4, 'sad':5, 'fear': 6}
    return emotion_dict[emotion]

In [55]:
single_target_lst = []
for i, segment in enumerate(range(len(annotation_20))):
    if annotation_20['Emotion'][segment].count(';') == 0:
        single_target_lst.append(i)

In [56]:
train_eda = annotation_20.iloc[single_target_lst]['EDA'].apply(sequence_padding)
train_temp = annotation_20.iloc[single_target_lst]['TEMP'].apply(sequence_padding)
target_v = annotation_20.iloc[single_target_lst]['Valence']
target_a = annotation_20.iloc[single_target_lst]['Arousal']
target_e = annotation_20.iloc[single_target_lst]['Emotion'].apply(make_emotion_vector)

In [57]:
train = torch.concat([torch.tensor([x for x in train_eda]), torch.tensor([x for x in train_temp])], dim=-1)
target_e = torch.tensor(target_e.values)

svm_clf = svm.SVC(kernel = 'linear')
X_train, X_test, y_train, y_test = train_test_split(train, target_e, test_size = 0.2, random_state=42)
svm_clf.fit(X_train, y_train)
svm_clf_predictions = svm_clf.predict(X_test)
print("Accuracy: {}%".format(svm_clf.score(X_test, y_test) * 100 ))
print("f1_score: {}%".format(f1_score(y_test, svm_clf_predictions, average='weighted')*100))
print(classification_report(svm_clf_predictions, y_test))

Accuracy: 85.61964146531567%
f1_score: 78.98650107045917%
              precision    recall  f1-score   support

           0       1.00      0.86      0.92      2566
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0

    accuracy                           0.86      2566
   macro avg       0.14      0.12      0.13      2566
weighted avg       1.00      0.86      0.92      2566



/home/arplab/project/paradeigma/multi_modal/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arplab/project/paradeigma/multi_modal/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arplab/project/paradeigma/multi_modal/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [59]:
svm_clf = svm.SVC(kernel = 'rbf', C = 10.0, gamma=0.1)
X_train, X_test, y_train, y_test = train_test_split(train, target_e, test_size = 0.2, random_state=42)
svm_clf.fit(X_train, y_train)
svm_clf_predictions = svm_clf.predict(X_test)
print("Accuracy: {}%".format(svm_clf.score(X_test, y_test) * 100 ))
print("f1_score: {}%".format(f1_score(y_test, svm_clf_predictions, average='weighted')*100))
print(classification_report(svm_clf_predictions, y_test))

Accuracy: 85.54169914263446%
f1_score: 79.25242960693352%
              precision    recall  f1-score   support

           0       1.00      0.86      0.92      2555
           1       0.01      0.33      0.02         9
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.03      0.50      0.06         2
           6       0.00      0.00      0.00         0

    accuracy                           0.86      2566
   macro avg       0.15      0.24      0.14      2566
weighted avg       0.99      0.86      0.92      2566



/home/arplab/project/paradeigma/multi_modal/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arplab/project/paradeigma/multi_modal/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arplab/project/paradeigma/multi_modal/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [60]:
gnb_clf = GaussianNB()
X_train, X_test, y_train, y_test = train_test_split(train, target_e, test_size = 0.2, random_state=32)
gnb_clf.fit(X_train, y_train)
gnb_clf_prediction = gnb_clf.predict(X_test)
print("Accuracy: {}%".format(gnb_clf.score(X_test, y_test)))
print("f1_score: {}%".format(f1_score(y_test, gnb_clf_prediction, average='weighted')))
print(classification_report(y_test, gnb_clf_prediction))

Accuracy: 0.10795011691348402%
f1_score: 0.16569264785475946%
              precision    recall  f1-score   support

           0       0.87      0.10      0.18      2232
           1       0.13      0.08      0.10       226
           2       0.02      0.83      0.04        36
           3       0.02      0.07      0.03        14
           4       0.00      0.00      0.00        24
           5       0.02      0.08      0.03        26
           6       0.01      0.38      0.01         8

    accuracy                           0.11      2566
   macro avg       0.15      0.22      0.06      2566
weighted avg       0.77      0.11      0.17      2566



In [61]:
ran_clf = RandomForestClassifier(n_estimators=64, max_depth=32,random_state=0)
X_train, X_test, y_train, y_test = train_test_split(train, target_e, test_size = 0.2, random_state=42)
ran_clf.fit(X_train,y_train)
ran_clf_prediction = ran_clf.predict(X_test)
print("Accuracy: {}%".format(ran_clf.score(X_test, y_test)))
print("f1_score: {}%".format(f1_score(y_test, ran_clf_prediction, average='weighted')))
print(classification_report(y_test, ran_clf_prediction))

Accuracy: 0.8565861262665627%
f1_score: 0.7908074047821705%
              precision    recall  f1-score   support

           0       0.86      1.00      0.92      2197
           1       1.00      0.00      0.01       268
           2       0.00      0.00      0.00        28
           3       0.00      0.00      0.00        11
           4       0.00      0.00      0.00        27
           5       0.00      0.00      0.00        30
           6       0.00      0.00      0.00         5

    accuracy                           0.86      2566
   macro avg       0.27      0.14      0.13      2566
weighted avg       0.84      0.86      0.79      2566



/home/arplab/project/paradeigma/multi_modal/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arplab/project/paradeigma/multi_modal/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arplab/project/paradeigma/multi_modal/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [62]:
tree_clf = DecisionTreeClassifier()  
X_train, X_test, y_train, y_test = train_test_split(train, target_e, test_size = 0.2, random_state=42)
tree_clf.fit(X_train,y_train)
tree_clf_prediction = tree_clf.predict(X_test)
print("Accuracy: {}%".format(tree_clf.score(X_test, y_test)))
print("f1_score: {}%".format(f1_score(y_test, tree_clf_prediction, average='weighted')))
print(classification_report(y_test, tree_clf_prediction))

Accuracy: 0.7470771628994544%
f1_score: 0.7503755837689581%
              precision    recall  f1-score   support

           0       0.86      0.86      0.86      2197
           1       0.14      0.13      0.14       268
           2       0.03      0.04      0.03        28
           3       0.00      0.00      0.00        11
           4       0.00      0.00      0.00        27
           5       0.00      0.00      0.00        30
           6       0.00      0.00      0.00         5

    accuracy                           0.75      2566
   macro avg       0.15      0.15      0.15      2566
weighted avg       0.75      0.75      0.75      2566



In [63]:
line_fitter = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(train, target_v, test_size = 0.2, random_state=30)
line_fitter.fit(X_train, y_train)
line_fitter_predictions = line_fitter.predict(X_test)
print("r2_score_Valence : ", line_fitter.score(X_test, y_test))
X_train, X_test, y_train, y_test = train_test_split(train, target_a, test_size = 0.2, random_state=30)
line_fitter.fit(X_train, y_train)
line_fitter_predictions = line_fitter.predict(X_test)
print("r2_score_Arousal : ", line_fitter.score(X_test, y_test))

r2_score_Valence :  -0.010923784295159278
r2_score_Arousal :  0.12036567706463785
